# Análise exploratória

## Carregamento e preparação dos dados

### Obtenção e carregamento de dados  

In [ ]:
#importação de bibliotecas
import pandas as pd

In [ ]:
# Importação dos dados gerais
df_dataset_cleaned = pd.read_csv('../Data/Normalization/data_cleaned.csv', sep=';', encoding='utf-8-sig')

# Importação dos dados para mesclagem
df_to_merge = pd.read_csv('../Data/Normalization/Course/normalization_courses_with_translated.csv', sep=';', encoding='utf-8-sig')

# Carregar dicionário de dados
df_dictionary = pd.read_csv("../Data/Dictionaries/dictionary_course_area.csv", encoding="utf-8", sep=";", on_bad_lines='skip')

### Informações sobre o dataset principal

In [ ]:
#Exibir informações sobre o dataframe
df_dataset_cleaned.info()

#Exibir as primeiras linhas do dataframe
df_dataset_cleaned.describe()

### Visualização dos dados principais

In [ ]:
#Exibir as primeiras linhas do dataframe
df_dataset_cleaned.head()

### Definição de índices do dataset principal

In [ ]:
# Colocar a coluna 'survey_Id' na primeira posição
df_dataset_cleaned = df_dataset_cleaned[['survey_Id'] + [col for col in df_dataset_cleaned.columns if col != 'survey_Id']]

# Definir o índice do dataframe como a coluna 'survey_Id'
df_dataset_cleaned.set_index('survey_Id', inplace=True)

# Exibir as primeiras linhas do dataframe
df_dataset_cleaned.head()

### Informações sobre o dataset para mesclagem

In [ ]:
#Exibir informações sobre o dataframe
df_to_merge.info()

#Exibir as primeiras linhas do dataframe
df_to_merge.describe()

### Visualização dos dados de mesclagem

In [ ]:
# Exibir as primeiras linhas do dataframe
df_to_merge.head()

### Definição de índices do dataset de mesclagem

In [ ]:
# Colocar a coluna 'survey_Id' na primeira posição
df_to_merge = df_to_merge[['survey_Id'] + [col for col in df_to_merge.columns if col != 'survey_Id']]

# Definir o índice do dataframe como a coluna 'survey_Id'
df_to_merge.set_index('survey_Id', inplace=True)

# Exibir as primeiras linhas do dataframe
df_to_merge.head()



## Mesclagem dos datasets

### Combinar os dataframes

In [ ]:
# Combinar os dois dataframes com base na coluna 'survey_Id'
df_combined = df_dataset_cleaned.merge(df_to_merge[['D9_3_1_O_college_area_other_translate_es', 'D9_3_2_O_master_area_other_translate_es', 'D9_3_3_O_doctorate_area_other_translate_es', 'D9_3_4_O_postdoc_area_other_translate_es']], left_index=True, right_index=True, how='inner')

# Exibir as primeiras linhas do novo dataframe
df_combined.head()

### Ordenar colunas

In [ ]:
# dicionário que mapeia cada coluna-origem para sua correspondente de tradução
pairs = {
    'D9_3_1_O_college_area_other':    'D9_3_1_O_college_area_other_translate_es',
    'D9_3_2_O_master_area_other':     'D9_3_2_O_master_area_other_translate_es',
    'D9_3_3_O_doctorate_area_other':  'D9_3_3_O_doctorate_area_other_translate_es',
    'D9_3_4_O_postdoc_area_other':    'D9_3_4_O_postdoc_area_other_translate_es',
}

# pega a lista atual de colunas
cols = list(df_combined.columns)

# para cada par, "move" a coluna de tradução para logo depois da coluna-origem
for base_col, trans_col in pairs.items():
    if base_col in cols and trans_col in cols:
        # retira a coluna de tradução da posição atual
        cols.remove(trans_col)
        # insere-a imediatamente após a coluna-origem
        insert_pos = cols.index(base_col) + 1
        cols.insert(insert_pos, trans_col)

# reorganiza o DataFrame segundo a nova ordem de colunas
df_combined = df_combined[cols]

# opcional: conferir o resultado
df_combined.head()

### Remover colunas desnecessárias

In [ ]:
#Remove as colunas 'D9_3_1_O_college_area_other', 'D9_3_2_O_master_area_other', 'D9_3_3_O_doctorate_area_other', 'D9_3_4_O_postdoc_area_other' do dataframe
df_combined.drop(columns=['D9_3_1_O_college_area_other', 'D9_3_2_O_master_area_other', 'D9_3_3_O_doctorate_area_other', 'D9_3_4_O_postdoc_area_other'], inplace=True)

### Renomear colunas

In [ ]:
# Renomear colunas
df_combined.rename(columns={
    'D9_3_1_O_college_area_other_translate_es': 'D9_3_1_O_college_area_other',
    'D9_3_2_O_master_area_other_translate_es': 'D9_3_2_O_master_area_other',
    'D9_3_3_O_doctorate_area_other_translate_es': 'D9_3_3_O_doctorate_area_other',
    'D9_3_4_O_postdoc_area_other_translate_es': 'D9_3_4_O_postdoc_area_other'
}, inplace=True)

### Adicionar colunas de descrição das áreas

In [ ]:
#Cria o map de descrições
desc_map = df_dictionary.set_index('course_area_id')['course_area_description']

#Lista das 4 colunas de área
area_cols = [
    'D9_3_1_college_area',
    'D9_3_2_master_area',
    'D9_3_3_doctorate_area',
    'D9_3_4_postdoc_area'
]

#Insere só a descrição em ordem reversa
for col in reversed(area_cols):
    pos = df_combined.columns.get_loc(col)
    df_combined.insert(
        loc    = pos + 1,
        column = f'{col}_description',
        value  = df_combined[col].map(desc_map)
    )

#Verifica
df_combined.head()

### Exportar dataset mesclado

In [ ]:
# Exportar o DataFrame combinado para um arquivo CSV
df_combined.to_csv('../Data/Normalization/Course/data_course_normalized.csv', sep=';', encoding='utf-8-sig')